In [1]:
!pip install yfinance
!pip install "redshift-connector[full]" sqlalchemy-redshift
!pip install python-dotenv
!pip install psycopg2-binary


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
import os
from configparser import ConfigParser
import sqlalchemy as sa
from sqlalchemy.engine.url import URL
from sqlalchemy.exc import SQLAlchemyError, ResourceClosedError
import pandas as pd
from pathlib import Path
import yfinance as yf
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# This file is used by dotenv to load the db credentials
env_file = Path('.').resolve() / '.env'

In [ ]:
# DB configuration environment variables
user = os.getenv('USERNAME')
passw = os.getenv('PASSW')
host = os.getenv('HOST')
port = os.getenv('PORT')
dbname = os.getenv('DB_NAME')
schema = "norbermv_dev_coderhouse"

In [ ]:
### Defining some useful functions for handling the Redshift db connection.

In [ ]:
def build_conn_string(
    user: str, 
    passw: str, 
    host: str, 
    port: str, 
    dbname: str
) -> URL:
    conn_string = URL.create(
        drivername="postgresql",
        username=user,
        password=passw,
        host=host,
        port=port,
        database=dbname
    )
    return conn_string

def conn_to_db(conn_str: URL) -> tuple :
    # Create an engine with the connection string
    engine = sa.create_engine(conn_str)
    try:
        # Connect to the database
        conn = engine.connect()
        return conn, engine
    except SQLAlchemyError as e:
        print(f"Error connecting to the database: {e}")
        return None, None

In [ ]:
# Build the connection string, and connect to the DB
conn_str = build_conn_string(
    user, 
    passw, 
    host, 
    port,
    dbname
)

In [ ]:
conn, engine = conn_to_db(conn_str)

## Create The `stock_hist`  Table

In [ ]:
if conn is not None:
    try:
        # Use the connection to execute a DDL statement
        with conn.begin() as trans:
            conn.execute(
                f"""
                DROP TABLE IF EXISTS {schema}.stock_hist;
                CREATE TABLE {schema}.stock_hist (
                    Date TIMESTAMP,
                    open_price FLOAT,
                    High FLOAT,
                    Low FLOAT,
                    Close FLOAT,
                    Volume INT,
                    Dividends FLOAT,
                    Stock_Splits FLOAT
                );
                """
            )
            # Commit the transaction to ensure DDL statement is executed
            trans.commit()
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        conn.close()
else:
    print("Failed to connect to the database.")


## Retrieving the data from the `yfinance` API
Using the yfinance API to fetch historical stock data for Apple Inc. (ticker symbol: AAPL)

In [ ]:
# Create a Ticker object for Apple Inc.
aapl = yf.Ticker("AAPL")

In [ ]:
# Get historical market data for Apple
aapl_hist = aapl.history(period="1y")

# Show meta information about the history for Apple (requires history() to be called first)
aapl_history_metadata = aapl.history_metadata

In [ ]:
# Rename the aapl_hist DataFrame columns to match the column names in the 'stock_hist' table
aapl_hist.columns = ['open_price', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock_Splits']

In [ ]:
aapl_hist.head()

In [ ]:
conn, engine = conn_to_db(conn_str)

In [ ]:
if conn is not None:
    try:
        with conn.begin() as trans:
            for index, row in aapl_hist.iterrows():
                sql = f"""
                INSERT INTO {schema}.stock_hist (Date, open_price, High, Low, Close, Volume, Dividends, Stock_Splits)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
                """
                conn.execute(sql, (index, row['open_price'], row['High'], row['Low'], row['Close'], row['Volume'], row['Dividends'], row['Stock_Splits']))
            # Commit the transaction
            trans.commit()
    except Exception as e:
        if isinstance(e, ResourceClosedError):
            print("Failed to connect to the database")
        else:
            print(f"An error occurred: {e}")
    finally:
        conn.close()

In [145]:
if conn is not None:
    try:
        with conn.begin() as trans:
            for index, row in aapl_hist.iterrows():
                sql = f"""
                INSERT INTO {schema}.stock_hist (Date, open_price, High, Low, Close, Volume, Dividends, Stock_Splits)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
                """
                conn.execute(sql, (index, row['open_price'], row['High'], row['Low'], row['Close'], row['Volume'], row['Dividends'], row['Stock_Splits']))
            # Commit the transaction
            trans.commit()
    except Exception as e:
        if isinstance(e, ResourceClosedError):
            print("Failed to connect to the database")
        else:
            print(f"An error occurred: {e}")
    finally:
        conn.close()

Failed to connect to the database
